In [25]:
import requests
from bs4 import BeautifulSoup
import csv

# 상명대학교 전자도서관 베스트 셀러 URL
URL="https://libebook.smu.ac.kr/FxLibrary/product/list/?category_type=book&cateopt=best" 

# 총 페이지 수 추출 함수
def get_page_count():
    result = requests.get(URL)

    soup = BeautifulSoup(result.text, "html.parser")

    count = len(soup.select("div.paging span a"))+1
    
    return count

# 책 정보 추출 함수
def extract_book(html):
    category=html.select_one("div.category span").text
    
    subject=html.select_one("div.subject a").text
    
    writer=html.select("div.info ul.i1 a")[0].text
    
    publisher=html.select("div.info ul.i1 a")[1].text
    
    date=html.select("div.info ul.i1 li")[2].text
    
    checkout=html.select("div.info ul.i2 li")[2].text
    
    like=html.select("div.info ul.i2 li")[3].text
    
    return {
        'category': category,
        'subject': subject,
        'writer':writer,
        'publisher':publisher,
        'date':date,
        'checkout':checkout,
        'like':like
    }

# 각 페이지마다 책 정보 추출 함수
def extract_books(page_count):
    books = []
    
    for page in range(page_count):
    
        print(f"Scrapping page {page+1}")
        
        rs = requests.get(f"https://libebook.smu.ac.kr/FxLibrary/product/list/?itemdv=1&sort=3&page={page+1}&itemCount=20&pageCount=10&category=&middlecategory=&cateopt=best&group_num=recommand&catenavi=main&category_type=book&searchoption=&keyoption=&keyoption2=&keyword=&listfilter=all_list&selectview=list_on&searchType=&name=&publisher=&author=&terminal=")
        
        soup = BeautifulSoup(rs.text, "html.parser")
        
        results=soup.find_all("li",class_="item")
        
        results=results[0:20]
        
        for rs in results:
            
            book = extract_book(rs)
            
            books.append(book)
            
    return books

# CSV 저장
page_count=get_page_count()

books=extract_books(page_count)

file=open("books.csv",mode="w")

write=csv.writer(file)

write.writerow(["category", "subject", "writer", "publisher","date","checkout","like"])

for book in books:
    
    write.writerow(list(book.values()))

Scrapping page 1
Scrapping page 2
Scrapping page 3
Scrapping page 4
Scrapping page 5


In [30]:
import pandas as pd

df=pd.read_csv("books.csv",encoding="cp949")

In [33]:
df

,category,subject,writer,publisher,date,checkout,like
0,[경제/비즈니스],트렌드 코리아 2023,"김난도, 전미영, 최지혜, 이수진, 권정윤, 이준영, 이향은, 한다혜, 이혜원, 추예린",미래의창,2022-10-06,누적대출 137,추천 1
1,[자연/과학],물고기는 존재하지 않는다,룰루 밀러,곰출판,2022-01-19,누적대출 168,추천 2
2,[문학],"오늘 밤, 세계에서 이 사랑이 사라진다 해도",이치조 미사키,모모,2021-10-21,누적대출 99,추천 1
3,[문학],불편한 편의점,김호연,나무옆의자,2022-02-22,누적대출 137,추천 0
4,[경제/비즈니스],역행자,자청,웅진지식하우스,2022-06-13,누적대출 170,추천 0
...,...,...,...,...,...,...,...
95,[문학],천 개의 파랑,천선란,허블,2020-08-19,누적대출 27,추천 2
96,[문학],초월하는 세계의 사랑,"우다영 , 조예은 , 문보영 , 심너울 , 박서련",허블,2022-05-03,누적대출 8,추천 0
97,[인문],"출근길엔 니체, 퇴근길엔 장자",필로소피 미디엄,한국경제신문(한경BP),2022-11-28,누적대출 7,추천 0
98,[장르문학],치우천왕기 4,이우혁,엘릭시르,2011-05-13,누적대출 27,추천 0
